In [40]:
## imporrt libarary 
import os
import pandas as pd

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
#from google.cloud import aiplatform
# from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [42]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [43]:
project_id = 'dtidsus'
dataset_id = 'capstone'
table_id = 'data_bank_marketing_campaign'
region = 'us-central1'
bucket_name = 'modul4'
#blob_name = 'ilham/german_dataset.csv'
blob_name = 'theresia/data_bank_marketing_campaign.csv'

In [44]:
client = bigquery.Client(project='dtidsus')

In [25]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    #data_capstone = bucket.blob('ilham/german_dataset.csv')
    data_capstone = bucket.blob('theresia/data_bank_marketing_campaign.csv')
    data_capstone.upload_from_filename('data_bank_marketing_campaign.csv')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [45]:
client = bigquery.Client('dtidsus')

In [46]:
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

In [19]:
#pip install db-dtypes

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip show db-dtypes


Name: db-dtypes
Version: 1.3.1
Summary: Pandas Data Types for SQL systems (BigQuery, Spanner)
Home-page: https://github.com/googleapis/python-db-dtypes-pandas
Author: The db-dtypes Authors
Author-email: googleapis-packages@google.com
License: 
Location: /opt/anaconda3/envs/purwadhika/lib/python3.12/site-packages
Requires: numpy, packaging, pandas, pyarrow
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [47]:
df = query_job.result().to_dataframe()

/opt/anaconda3/envs/purwadhika/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [10]:
df.head()

,age,job,balance,housing,loan,contact,month,campaign,pdays,poutcome,deposit
0,18,student,608,False,False,cellular,aug,1,-1,unknown,True
1,18,student,108,False,False,cellular,aug,1,-1,unknown,True
2,18,student,108,False,False,cellular,feb,1,183,success,True
3,18,student,348,False,False,cellular,may,4,-1,unknown,True
4,18,student,608,False,False,cellular,nov,1,93,success,True


In [15]:




#cleansing 
result = df.drop(['ID'], axis = 1)
result.head()

,LIMIT_BAL,MARRIAGE,EDUCATION,SEX,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,TARGET
0,50000,married,Univ,male,23,131895,132154,129237,1
1,50000,married,Univ,male,24,56626,58512,61926,1
2,40000,married,Univ,male,25,12961,12029,11738,1
3,30000,single,Univ,male,26,28949,30319,29400,0
4,30000,married,Univ,male,26,0,0,0,1


In [17]:
result.isna().sum()

LIMIT_BAL    0
MARRIAGE     4
EDUCATION    0
SEX          0
AGE          4
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
TARGET       0
dtype: int64

In [19]:
result  = result.dropna()

In [20]:
categorical_cols = result.select_dtypes(include=['object']).columns

# Create a dictionary to store label encoders for each column
encoders = {}

# Iterate through categorical columns
for col in categorical_cols:
    # Create a LabelEncoder object for each column
    encoder = LabelEncoder()

    # Fit and transform the column
    result[col] = encoder.fit_transform(result[col])

    # Store the encoder in the dictionary for later use
    encoders[col] = encoder


# Preprocessing
y = result['TARGET']  
X = result.drop(['TARGET'], axis=1)  

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Evaluation
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

# Save Model
model_filename = "model.pkl"
pickle.dump(model, open(model_filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.68      0.76      0.72      1980
         1.0       0.58      0.48      0.52      1344

    accuracy                           0.65      3324
   macro avg       0.63      0.62      0.62      3324
weighted avg       0.64      0.65      0.64      3324



c:\Users\cbcbx\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##upload the model

In [48]:
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    #blob_model = bucket.blob('ilham/model/model.pkl')
    #blob_model = bucket.blob('theresia/model/model.pkl')
    #blob_model.upload_from_filename('model.pkl')
    blob_model = bucket.blob('theresia/model/model.pkl')  # Rename to model.pkl
    blob_model.upload_from_filename('catboost_model2.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [14]:
pip install google-cloud-aiplatform


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [49]:
import joblib

#model = joblib.load('model.pkl')


In [ ]:
#test_input = [[18,student,,608, False	False cellular	aug 1 -1	unknown	]]  # Provide sample input
##prediction = model.predict(test_input)
#print(prediction)

In [50]:
joblib.dump(model, 'model.pkl')

['model.pkl']

In [29]:
from google.cloud import aiplatform

In [30]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [31]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
    display_name='theresia_model_000',
    artifact_uri="gs://modul4/theresia/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

Creating Model
Create Model backing LRO: projects/41965541199/locations/us-central1/models/4950816086363734016/operations/440037094740459520
Model created. Resource name: projects/41965541199/locations/us-central1/models/4950816086363734016@1
To use this Model in another session:
model = aiplatform.Model('projects/41965541199/locations/us-central1/models/4950816086363734016@1')


In [32]:
endpoint = aiplatform.Endpoint.create(
    display_name="theresia-endpoint-000",
    project='dtidsus',
    location='us-central1',
)

Creating Endpoint
Create Endpoint backing LRO: projects/41965541199/locations/us-central1/endpoints/7402274017513570304/operations/1097562640336551936
Endpoint created. Resource name: projects/41965541199/locations/us-central1/endpoints/7402274017513570304
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/41965541199/locations/us-central1/endpoints/7402274017513570304')


In [36]:
min_replica_count: int = 1
max_replica_count: int = 2

In [37]:
endpoint.deploy( 
    model=model,
    deployed_model_display_name='theresia_model_000',
    machine_type='e2-standard-2',
    min_replica_count=min_replica_count,
    max_replica_count=max_replica_count,
    sync=True, 
)

Deploying Model projects/41965541199/locations/us-central1/models/4950816086363734016 to Endpoint : projects/41965541199/locations/us-central1/endpoints/7402274017513570304
Deploy Endpoint model backing LRO: projects/41965541199/locations/us-central1/endpoints/7402274017513570304/operations/3697265525236170752


FailedPrecondition: 400 Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FDeploymentResourcePool&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FDeploymentResourcePool%22%0Aresource.labels.deployment_resource_pool_id%3D%22internal_1998854964692123648%22%0Aresource.labels.location%3D%22us-central1%22. 9: Model server exited unexpectedly. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=41965541199&resource=aiplatform.googleapis.com%2FDeploymentResourcePool&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FDeploymentResourcePool%22%0Aresource.labels.deployment_resource_pool_id%3D%22internal_1998854964692123648%22%0Aresource.labels.location%3D%22us-central1%22.

#using endpoint

In [45]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/2241324766407426048"
REGION = 'us-central1'



In [46]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)

PREDICTION: Prediction(predictions=[0.0], deployed_model_id='6724688981780332544', metadata=None, model_version_id='1', model_resource_name='projects/41965541199/locations/us-central1/models/4379281146152747008', explanations=None)


#manual file

In [28]:
try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob('ilham/model/model.pkl')
    blob_model.download_to_filename('model_new.pkl')

    print ("download model succeeded")
except:
    raise TypeError("An exception occurred")

download model succeeded


In [29]:
model_load = pickle.load(open('model_new.pkl','rb'))

In [30]:
y_predict = model_load.predict(X_test)

In [31]:
y_predict

array([0., 0., 0., ..., 0., 0., 1.])